In [1]:
!pip install flask
!pip install flask_restful
!pip install requests-oauthlib==1.1.0
!pip install oauthlib==2.1.0
!pip install xlrd
!pip install xlwt

### Basic Setup

In [2]:
from flask import Flask, render_template, request, redirect, url_for, Response
from flask_restful import Resource, Api, reqparse
import sys

from fitbit.api import Fitbit
from oauthlib.oauth2.rfc6749.errors import MismatchingStateError, MissingTokenError, InvalidClientError

import pandas as pd 
import numpy as np
import datetime
from datetime import timedelta  
import time
import threading

from fetchHelper import DatabaseFactory

### Flask Web Server setting
---
Constructing Fitbit Application Client. 

1. CLIENT_ID: client id of fitbit app (see manage app in dev.fitbit.com)
2. CLIENT_SECRET: client secret of fitbit app
3. REDIRECT_URI: redirection url set on fitbit app 

In [3]:
app = Flask(__name__)
api = Api(app)

CLIENT_ID = r'22BDG9'
CLIENT_SECRET = r'c8f4b83a7f2b20602e4dc1006221395a'
REDIRECT_URI = "https://fitbit.run-us-west1.goorm.io/auth"

class OAuth2Server:
    def __init__(self, client_id, client_secret, redirect_uri=REDIRECT_URI):
        self.success_html = """
            <h1>You are now authorized to access the Fitbit API!</h1>
            <br/><h3>You can close this window</h3>"""
        
        self.failure_html = """
            <h1>ERROR: %s</h1><br/><h3>You can close this window</h3>%s"""

        self.fitbit = Fitbit(
            client_id,
            client_secret,
            redirect_uri=redirect_uri,
            timeout=10,
        )
        
        print(redirect_uri)
    
    def auth_url(self):
        print(self.fitbit.client.authorize_token_url())
        return self.fitbit.client.authorize_token_url()
    
    def browser_authorize(self):
        app.run(host="0.0.0.0", threaded=True)

server = OAuth2Server(CLIENT_ID, CLIENT_SECRET)
auth2_client = None
auth2_client_new = None


https://fitbit.run-us-west1.goorm.io/auth


In [4]:
@app.route('/', methods=['GET'])
def index():
    url, _ = server.auth_url()
    print(url)
    
#     return url
    return redirect(url, code=301)

@app.route('/auth', methods=['GET'])
def auth():
    code = request.args.get('code', type=
                           str)
    state = request.args.get('state', type=str)
    error = request.args.get('error', type=str)
    
#     return code
    if code:
        try:
            token = server.fitbit.client.fetch_access_token(code)
            ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
            REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
            
            ##Build Auth Clients
            auth2_client = Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')

            #NEed this seeparately because version 1 doesn't give sleep breakdown
            auth2_client_new = Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')
            auth2_client_new.API_VERSION = 1.2
            token = Token(ACCESS_TOKEN, REFRESH_TOKEN)
            tokenQueue.put(token)
            tokenQueue.printAll()
            

            dbFactory = DatabaseFactory('data/database_main.xls', auth2_client, auth2_client_new)
            database = dbFactory.connectAndLoadDb()
            
            
            run_check(database, dbFactory)
            return "succeed"
        except MissingTokenError:
            error = server._fmt_failure(
                'Missing access token parameter.</br>Please check that '
                'you are using the correct client_secret')
        except MismatchingStateError:
            error = server._fmt_failure('CSRF Warning! Mismatching state')
#         except InvalidClientError:
#             return code


In [5]:
tokenQueue = TokenQueue();

server.browser_authorize()

ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])

print(ACCESS_TOKEN, REFRESH_TOKEN)

NameError: name 'TokenQueue' is not defined

## Helpers
---
### Helpers for user token queue

In [ ]:
from queue import Queue

In [ ]:
class Token:
    def __init__(self, token, refreshToken):
        self.token = token
        self.refreshToken = refreshToken

    def __str__(self):
        return "Token: {}, Refresh Token: {}".format(self.token, self.refreshToken)

class TokenQueue:
    def __init__(self):
        self.queue = Queue()
        self.waitQueue = Queue()
        
    def put(self, token):
        self.queue.put_nowait(token)
    
    def get(self):
        self.queue.get_nowait()
        
    def getWait(self):
        self.waitQueue.get_nowait()
    
    def putWait(self, token):
        self.waitQueue.put_nowait(token)
    
    def printAll(self):
        for i in range(self.queue.qsize()):
            t = self.queue.get_nowait()
            print(t)
            self.queue.put_nowait(t)
            
def run_check(database, dbFactory):
    threading.Timer(60.0, run_check, [database, dbFactory]).start()
    database = dbFactory.fetchData(database)
    dbFactory.writeDbToExcelFile(database)
